## 환경 설정

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=7892864b6298267b3e9077c4eb1d123868a99814422fbd35102981879df4ed8f
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5wv4_pjv/kobert-tokenizer_6ca4253ef58a48c3bb51bbf53e0b454d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5wv4_pjv/kobert-tokenizer

In [ ]:
!pip install mecab-python3
!pip install konlpy
!pip install tensorflow-text
!pip install soynlp
!pip install -v python-mecab-ko
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled ke

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
from konlpy.tag import Kkma
from soynlp.normalizer import *
from soynlp.tokenizer import LTokenizer
from konlpy.tag import Komoran
from transformers import BertTokenizer
import mecab
import sentencepiece as spm

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

## 데이터

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/WCRC/final_train_df.csv')
val_df = pd.read_csv('/content/drive/MyDrive/WCRC/final_val_df.csv')

In [ ]:
with open('/content/drive/MyDrive/WCRC/018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.json', encoding="utf-8") as f:
    labeled_data = json.load(f)
with open('/content/drive/MyDrive/WCRC/018.감성대화/Validation_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Validation.json', encoding="utf-8") as f:
    validation_data = json.load(f)

## 상황 분류 모델 예측

In [ ]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    """
    BERT 스타일의 데이터 변환 클래스.

    Parameters
    ----------
    tokenizer : BERTTokenizer
        문장을 토큰화하는 토크나이저.
    max_seq_length : int
        문장의 최대 시퀀스 길이.
    vocab : BERTVocab
        토큰 변환을 위한 어휘.
    pad : bool, default True
        문장을 최대 길이로 패딩할지 여부.
    pair : bool, default True
        문장 또는 문장 쌍을 변환할지 여부.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """
        문장 쌍 또는 단일 문장에 대한 변환 수행.

        Parameters
        ----------
        line: tuple of str
            입력 문자열. 문장 쌍의 경우 입력은 2개의 문자열로 이루어진 튜플이며:
            (text_a, text_b). 단일 문장의 경우 입력은 단일 문자열로 이루어진 튜플:
            (text_a,).

        Returns
        -------
        np.array: 'int32' 형식의 입력 토큰 ID, 형태 (batch_size, seq_length)
        np.array: 'int32' 형식의 유효 길이, 형태 (batch_size,)
        np.array: 'int32' 형식의 입력 토큰 타입 ID, 형태 (batch_size, seq_length)
        """

        # 유니코드로 변환
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # 'tokens_a' 및 'tokens_b'를 수정하여 총 길이가 지정된 길이보다 작아지도록 함
            # [CLS], [SEP], [SEP]에 해당하는 "- 3" 고려
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # [CLS] 및 [SEP]에 해당하는 "- 2" 고려
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # 문장의 길이
        valid_length = len(input_ids)

        if self._pad:
            # 시퀀스 길이까지 제로 패딩
            padding_length = self._max_seq_length - valid_length
            # 나머지는 패딩 토큰으로 채움
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 하이퍼 파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 12,   # 상황 클래스 수로 조정 클래스 12개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 저장한 모델 불러오기
loaded_model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
checkpoint_path = '/content/drive/MyDrive/WCRC/situation_model.pth'  # 모델 체크포인트 파일 경로
loaded_model.load_state_dict(torch.load(checkpoint_path))
loaded_model.eval()

# 예측할 문장
input_sentence = "직장 동료와 상사들이랑 친하게 지내야겠어."

# 입력 문장을 BERT 모델의 입력 형식으로 변환
transform = BERTSentenceTransform(tokenizer, max_seq_length=max_len, vocab=vocab, pad=True, pair=False)
input_data = transform([input_sentence])

# 토큰 ID, 유효 길이, 토큰 타입 ID를 추출
input_token_ids, input_valid_length, input_segment_ids = input_data

# 텐서로 변환
input_token_ids = torch.tensor([input_token_ids], dtype=torch.long).to(device)
input_valid_length = torch.tensor(input_valid_length, dtype=torch.long).to(device)  # 스칼라 값으로 전달
input_segment_ids = torch.tensor([input_segment_ids], dtype=torch.long).to(device)

# 예측 수행
with torch.no_grad():
    input_valid_length = torch.tensor([input_valid_length], dtype=torch.long).to(device)  # 스칼라 값으로 전달
    output = loaded_model(input_token_ids, input_valid_length, input_segment_ids)
    predicted_situation_label = torch.argmax(output, dim=1).item()

print("Predicted Label:", predicted_situation_label)

Predicted Label: 1


## 감정 분류 모델 예측

In [ ]:
import torch.nn as nn
from transformers import DistilBertModel

class JointDistilBertModel(nn.Module):
    def __init__(self, num_intents, dropout_rate=0.4):
        super(JointDistilBertModel, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.intent_classifier = nn.Linear(self.distilbert.config.dim, num_intents)

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # First token of the last hidden state
        pooled_output = self.dropout(pooled_output)
        intent_logits = self.intent_classifier(pooled_output)
        return intent_logits
num_intents = 6

model = JointDistilBertModel(6)
model.load_state_dict(torch.load('/content/drive/MyDrive/WCRC/emotion_model.pth'))
model.eval()

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

JointDistilBertModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [ ]:
def preprocess_input(text, tokenizer, max_length=512):
    # 입력 데이터 토큰화
    encoded_input = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        return_attention_mask=True,
        truncation=True
    )
    input_ids = torch.tensor([encoded_input['input_ids']], dtype=torch.long)
    attention_mask = torch.tensor([encoded_input['attention_mask']], dtype=torch.long)
    return input_ids, attention_mask

In [ ]:
def index_to_emotion(index):
    # 감정 매핑
    emotion_labels = {0: "불안", 1: "분노", 2: "상처", 3: "슬픔", 4: "당황", 5: "기쁨"}
    return emotion_labels.get(index, "Unknown")

In [ ]:
def predict_emotion(text, model, tokenizer):
    input_ids, attention_mask = preprocess_input(text, tokenizer)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs, dim=1)

    # Convert the prediction to the corresponding emotion label
    # Assuming you have a function or a dictionary to map prediction indices to emotion labels
    # emotion_label = index_to_emotion(prediction.item())  # Define this mapping as per your dataset
    emotion_label = int(prediction.item())

    return emotion_label

In [ ]:
# 예시 텍스트
example_text = '나 요즘 매일 퇴사를 고민 하고 있는데 너무 힘들다.'

# 예측
predicted_emotion_label = predict_emotion(example_text, model, tokenizer)
print(f"당신의 감정은: {predicted_emotion_label}")

## 답변 생성

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# '사람문장_명사' 열의 문장을 공백을 기준으로 단어로 나누어 리스트로 변환
train_df['사람문장_명사'] = train_df['사람문장_명사'].apply(lambda x: x.split() if isinstance(x, str) else [])

# 명사 추출 함수 정의
def extract_nouns(sentence):
    okt = Okt()
    nouns = okt.nouns(sentence)
    return ' '.join(nouns)

def get_most_similar_response(sentence, dataframe, situation_keyword, emotion_category):
    # 상황키워드와 감정대분류에 해당하는 행들만 필터링
    filtered_df = dataframe[(dataframe['상황키워드'] == situation_keyword) & (dataframe['감정_대분류'] == emotion_category)]

    # 명사 추출
    input_nouns = extract_nouns(sentence)

    # '사람문장_명사' 값을 가져와서 문장 벡터화
    vectorizer = CountVectorizer()
    # 리스트를 문자열로 변환하여 공백으로 구분된 단어들의 문자열로 만듦
    sentence_vector = vectorizer.fit_transform(filtered_df['사람문장_명사'].apply(lambda x: ' '.join(x)))

    # 입력 문장을 벡터화
    input_vector = vectorizer.transform([input_nouns])

    # 코사인 유사도 계산
    similarities = cosine_similarity(input_vector, sentence_vector).flatten()

    if similarities.size == 0:
        return "죄송합니다. 다시 입력해주세요."

    # 가장 유사한 행의 인덱스 가져오기
    most_similar_index = similarities.argmax()

    # 가장 유사한 답변 반환
    return filtered_df.iloc[most_similar_index]['시스템문장']

# 예시 문장
input_sentence = "남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네"

# 함수 호출
response = get_most_similar_response(input_sentence, train_df, predicted_situation_label, predicted_emotion_label)
print(response)

결혼 하면 어떤 점에서 그렇게 느끼게 만드나요?


In [ ]:
# 예시 문장
input_sentence = "회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워."

# 함수 호출
response = get_most_similar_response(input_sentence, train_df, predicted_situation_label, predicted_emotion_label)
print(response)

큰 프로젝트를 혼자 하셔서 고민이 많겠네요.


In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# '사람문장_명사' 열의 문장을 공백을 기준으로 단어로 나누어 리스트로 변환
# train_df['사람문장_명사'] = train_df['사람문장_명사'].apply(lambda x: x.split() if isinstance(x, str) else [])

# 명사 추출 함수 정의
def extract_nouns(sentence):
    okt = Okt()
    nouns = okt.nouns(sentence)
    return ' '.join(nouns)

def get_most_similar_response(sentence, dataframe, situation_keyword, emotion_category):
    # 상황키워드와 감정대분류에 해당하는 행들만 필터링
    filtered_df = dataframe[(dataframe['상황키워드'] == situation_keyword) & (dataframe['감정_대분류'] == emotion_category)]

    # 명사 추출
    input_nouns = extract_nouns(sentence)

    # '사람문장_명사' 값을 가져와서 문장 벡터화
    vectorizer = CountVectorizer()
    # 리스트를 문자열로 변환하여 공백으로 구분된 단어들의 문자열로 만듦
    sentence_vector = vectorizer.fit_transform(filtered_df['사람문장_명사'].apply(lambda x: ' '.join(x)))

    # 입력 문장을 벡터화
    input_vector = vectorizer.transform([input_nouns])

    # 코사인 유사도 계산
    similarities = cosine_similarity(input_vector, sentence_vector).flatten()

    if similarities.size == 0:
        return "죄송합니다. 다시 입력해주세요."

    # 가장 유사한 행의 인덱스 가져오기
    most_similar_index = similarities.argmax()

    # 가장 유사한 답변 반환
    response = filtered_df.iloc[most_similar_index]['시스템문장']
    return response

# 초기 대화 시작
print("안녕하세요! 상황과 감정에 관한 이야기를 나눠봐요.")
print("종료하려면 '종료'를 입력하세요.")

while True:
    # 사용자 입력 받기
    user_input = input("사용자: ")

    # 종료 조건 확인
    if user_input == '종료':
        print("대화를 종료합니다.")
        break

    # 이전에 예측된 상황과 감정에 대한 답변 가져오기
    response = get_most_similar_response(user_input, train_df, predicted_situation_label, predicted_emotion_label)

    # 답변 출력
    print("챗봇:", response)

안녕하세요! 상황과 감정에 관한 이야기를 나눠봐요.
종료하려면 '종료'를 입력하세요.
사용자: 이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.
챗봇: 실수하시다니 정말 죄송한 마음이 크겠어요.
사용자: 퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.
챗봇: 퇴근 후에도 일을 추가로 해서 만회하려고 하시는군요. 앞으로 어떤 점이 달라지기를 기대하시나요?
사용자: 내 능력이 부족한 거 같은데 그만 다녀야 될거같아.
챗봇: 능력을 올리려면 어떤 방법이 있을까요?
사용자: 회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.
챗봇: 큰 프로젝트를 혼자 하셔서 고민이 많겠네요.
사용자: 나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.
챗봇: 프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?
사용자: 상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.
챗봇: 상사가 옆을 지날 때마다 두려우시군요.
사용자: 종료
대화를 종료합니다.
